# Image plates of duckweed in growth assay experiment

Ensure the following before running anything in this notebook:
 - Plates with duckweed are loaded in the correct positions on the bedplate
 - The machine is switched on, connected to the computer and all axes have been homed. 
 - The camera tool is accessible and in position 1 on the Jubilee
 
 Once those steps are complete run the cells in this notebook in order

In [1]:
#Importing python libraries downloaded from the internet and those saved locally
import serial 
from serial.tools import list_ports
import yaml
import matplotlib
matplotlib.use('TkAgg') #This is a specific back end of matplotlib 
from matplotlib import pyplot as plt
import numpy as np
import os
import cv2 as cv2
import time
import pandas as pd
import glob
import json
import datetime
from datetime import date
from utils.MachineUtils import * #Local library
from utils.CameraUtils import * #Local library. #Note that this library requirs a package that can only be installed on raspbery pi. 
import utils.Plate_positions as pp #Local library

import ipywidgets as widgets
# from Ipython.display import display, Image
import threading

In [ ]:
def move_across_platewells(well_coord_list_of_dicts):
    for i, well in enumerate(well_coord_list_of_dicts):
        m.moveTo(x=well["x"], y=well["y"], z=10)
        ser.write(b'G4 P1000') 

def image_plate(plate_num):
    well_coord_list_of_dicts = pp.fetch_plate_wellpostions(plate_num, tool)
    for i, well in enumerate(well_coord_list_of_dicts):
        m.moveTo(x=well["x"], y=well["y"], z=10)
#         wellFrame, wellData = getSingleWell(getFrame(), minR=330, maxR=335) #Adjust focus
#         well = wellData[0]
#         print(well)
        f = getFrame()
        cv2.imwrite(f'{expt_dir}/{expt_id}_plate{plate_num}_well{well["well_id"]}_{date.today()}.jpg', f)

In [ ]:
def move_to_first_well(plates_to_image):
    first_plate = plates_to_image[0]
    well_coord_list_of_dicts = pp.fetch_plate_wellpostions(first_plate)
    for i, well in well_coord_list_of_dicts[0]:
        m.moveTo(x=well["x"], y=well["y"], z=10)

In [2]:
ports = serial.tools.list_ports.comports()
ser = serial.Serial('/dev/ttyACM0', 115200) #Port address and Baudrate (info transfer rate)
port = '/dev/ttyACM0'
m = MachineCommunication(port)
ser.write(b'G90\n') #Command to make sure the Jubilee is using absolute as opposed to relative positioning
m.toolChange(1)

### 1. Define output directory

In [ ]:
#Define directory to save image data for this experiment
data_dir = '/home/pi/Documents/Data'
expt_dir = data_dir + '/Test_expt'
expt_id = "test"
print("Image files will be saved to:")
print(expt_dir)


### 2. Option 1 - Read file to define plates to be imaged

In [ ]:

#Automated option: Define experiment set up file and read from that which plates are to be imaged. 

expt_setup_parent_dir = os.getcwd() # Default uses current working directory but you can replace with your own choice. 
expt_setup_dir = "TestExptDir" # Name of the folder to hold experiment data and metadata including the setup file
expt_setup_file_name = "TestFile.json" #Name for the experiment setup file (Metadata)

exp_setup_file_path = os.path.join(expt_setup_parent_dir, expt_setup_dir)
print(exp_setup_file_path)

In [ ]:
os.chdir(exp_setup_file_path)

with open(expt_setup_file_name) as datafile:
    expt_data = json.load(datafile)

# Turn samples list into a dataframe
sample_data = expt_data["sample_info"]
df = pd.DataFrame(sample_data)

unique_plates = list(df.Plate.unique())

def pull_last_number(n):
    return n[-1:]

plates_to_image = list(map(pull_last_number, unique_plates))
print(plates_to_image)


### 2. Option 2 - Manually define plates to be imaged

In [ ]:
# Manual option: Which plates do you want to image (numbers correspond to positions on bedplate. Options: 1,2,3,4,5)
plates_to_image = [1]

### 3. Focus camera
Run the cells below. It will move the camera over the first well of the first plate and then pop up a live camera window. Manually adjust the focus to the desired point by twisting the housing of the camera lens. Then hit the escape key to close the window

In [7]:
#Move camera over first well to be imaged
move_to_first_well(plates_to_image)

#Open a live video window
cap = cv2.VideoCapture(0) #Note that the index corresponding to your camera may not be zero but this is the most common default

while True:
    ret, frame = cap.read()
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c ==27: #27 is the built in code for ESC so press escape to close the window. 
        break 
        
cap.release()
cv2.destroyAllWindows()

### 3. Capture and save images

In [ ]:
for p in plates_to_image:
    image_plate(p)